In [ ]:
! pip install datasets
! pip install python-dotenv
! pip install openai
! pip install anthropic
! pip install tqdm


In [1]:
import os
os.getcwd()

'c:\\Users\\G25971483\\Desktop\\Projects\\LLM\\self_recognition'

In [2]:
from data import load_data, SOURCES, save_to_json, load_from_json
from models import (
    get_gpt_recognition_logprobs,
    get_model_choice,
    get_logprobs_choice_with_sources,
    get_gpt_score,
)

from math import exp
from pprint import pprint
from random import shuffle

from tqdm import tqdm
import time


c:\Users\G25971483\Desktop\Projects\LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def generate_score_results(dataset, model, starting_idx=0):
    SCORES = ["1", "2", "3", "4", "5"]

    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]
        for target_model in tqdm(SOURCES, desc="Processing target models", leave=False):
            summary = responses[target_model][key]
            response = get_gpt_score(summary, article, exact_model)
            result = {i.token: exp(i.logprob) for i in response if i.token in SCORES}
            for score in SCORES:
                if score not in result:
                    result[score] = 0

            results.append(
                {
                    "key": key,
                    "model": model,
                    "target_model": target_model,
                    "scores": result,
                }
            )

    return results

In [10]:
# responses["gpt4"]['06352019a19ae31e527f37f7571c6dd7f0c5da37']

In [5]:
for model in ["gpt4"]:
    print(model)
    results = generate_score_results("cnn", model, starting_idx=980)
    print(results)
    #save_to_json(results, f"individual_setting/score_results/cnn/{model}_results.json")
    #print(model)

gpt4


Processing keys: 100%|██████████| 20/20 [00:56<00:00,  2.83s/it]

[{'key': 'bd92eab23374d95ac4430e2e1a8bf7561c662309', 'model': 'gpt4', 'target_model': 'gpt4', 'scores': {'5': 0.7422421806885163, '4': 0.2573603335519957, '3': 0.0003956113769137268, '2': 1.7079717543507627e-06, '1': 9.457979189875667e-08}}, {'key': 'd84440effc748f9220c7dde068962e803f566bc7', 'model': 'gpt4', 'target_model': 'gpt4', 'scores': {'4': 0.6675366725947747, '5': 0.3296438004701316, '3': 0.0028128108306324767, '2': 6.298002741428309e-06, '1': 1.314580562613995e-07}}, {'key': 'e2866f596c85b0191ce812260d23855c70a01192', 'model': 'gpt4', 'target_model': 'gpt4', 'scores': {'4': 0.505405197574499, '5': 0.49409879100322596, '3': 0.000494314798402581, '2': 1.3698285115640545e-06, '1': 7.809061139023033e-08}}, {'key': '4ebd9195bc8c12272b6281a5a610a7b5fb49b8cd', 'model': 'gpt4', 'target_model': 'gpt4', 'scores': {'5': 0.5580828560942044, '4': 0.4411903032377108, '3': 0.0007239827440845187, '2': 2.445257802256994e-06, '1': 1.0231873939219596e-07}}, {'key': '4bd37aa7004cd1fdf1ccb506d215

In [7]:
print(results[0]) # 0.57 $ for 20 articles in individual comparison

{'key': 'bd92eab23374d95ac4430e2e1a8bf7561c662309', 'model': 'gpt4', 'target_model': 'gpt4', 'scores': {'5': 0.7422421806885163, '4': 0.2573603335519957, '3': 0.0003956113769137268, '2': 1.7079717543507627e-06, '1': 9.457979189875667e-08}}


# for Pairwise Comparison

In [4]:
# Only suitable for GPT models
def generate_gpt_logprob_results(
    dataset,
    model,
    starting_idx=0,
    detection_type="detection",
    comparison_type="comparison",
):
    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]

        source_summary = responses[model][key]
        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # Detection
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result[0].logprob)
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            # Comparison
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            # If the comparison asked "Which is worse?" then reverse the options
            if comparison_type == "comparison_with_worse":
                forward_choice = "1" if forward_choice == "2" else "2"
                backward_choice = "1" if backward_choice == "2" else "2"

            result["forward_comparison"] = forward_choice
            result["forward_comparison_probability"] = exp(forward_result[0].logprob)
            result["backward_comparison"] = backward_choice
            result["backward_comparison_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            results.append(result)
    return results


In [5]:
for model in ["gpt4"]:
    print(SOURCES)
    print(f"Starting {model}")
    results = generate_gpt_logprob_results(
        "cnn", model, comparison_type="comparison_with_worse", starting_idx=998
    )
    #save_to_json(results, f"results_with_worse/cnn/{model}_results.json")
    print(results)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting gpt4


Processing keys: 100%|██████████| 2/2 [04:59<00:00, 149.82s/it]

[{'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c', 'model': 'human', 'forward_detection': '1', 'forward_detection_probability': 0.7095165758036815, 'backward_detection': '1', 'detection_score': 0.566544069240114, 'forward_comparison': '1', 'forward_comparison_probability': 0.9991200103182194, 'backward_comparison': '2', 'backward_comparison_probability': 0.9978128770465099, 'self_preference': 0.9984664436823647}, {'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c', 'model': 'claude', 'forward_detection': '1', 'forward_detection_probability': 0.6985912162845513, 'backward_detection': '1', 'detection_score': 0.4960112491432166, 'forward_comparison': '1', 'forward_comparison_probability': 0.8605825976198661, 'backward_comparison': '2', 'backward_comparison_probability': 0.792832859043062, 'self_preference': 0.8267077283314641}, {'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c', 'model': 'gpt35', 'forward_detection': '1', 'forward_detection_probability': 0.758313166685928, 'backward_dete

In [6]:
results

[{'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c',
  'model': 'human',
  'forward_detection': '1',
  'forward_detection_probability': 0.7095165758036815,
  'backward_detection': '1',
  'detection_score': 0.566544069240114,
  'forward_comparison': '1',
  'forward_comparison_probability': 0.9991200103182194,
  'backward_comparison': '2',
  'backward_comparison_probability': 0.9978128770465099,
  'self_preference': 0.9984664436823647},
 {'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c',
  'model': 'claude',
  'forward_detection': '1',
  'forward_detection_probability': 0.6985912162845513,
  'backward_detection': '1',
  'detection_score': 0.4960112491432166,
  'forward_comparison': '1',
  'forward_comparison_probability': 0.8605825976198661,
  'backward_comparison': '2',
  'backward_comparison_probability': 0.792832859043062,
  'self_preference': 0.8267077283314641},
 {'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c',
  'model': 'gpt35',
  'forward_detection': '1',
  'forward_detection